In [18]:
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.model_selection  import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

In [19]:
df= pd.read_csv(r"long_format_df/visit_ft_long_udder.csv")
df = df.dropna()
print(df.shape)
print(df.head())


(43, 16)
    cow  vol_udder  sarea_udder   peri_udder  area_udder  circ_udder  \
5   778   7.435424  2315.909903  1204.187950     88234.0    0.768021   
11  868   3.687446  2171.163275  1118.388960     78328.0    0.787254   
12  915   9.792706  2299.151507  1202.593542     86760.0    0.755408   
13  922   8.601104  2022.001979  1169.234631     86290.0    0.794134   
15  929  10.490411  2178.960033  1221.785930     94098.0    0.789787   

    exc_udder      date           begin_time  days_in_milk device_name  \
5    0.576754  20231117  2023-11-17 14:02:00          54.0       VMS_2   
11   0.614288  20231117  2023-11-17 14:24:00          60.0       VMS_1   
12   0.561817  20231117  2023-11-17 12:33:00         246.0       VMS_1   
13   0.740400  20231117  2023-11-17 14:12:00         181.0       VMS_2   
15   0.697942  20231117  2023-11-17 15:15:00          14.0       VMS_2   

    lactation_number  interval_sec  yield is_incomplete    occ  
5                  5       21352.0  13.87     Un

In [25]:
y = df["yield"]
X=df[["sarea_udder", "vol_udder", 
          "days_in_milk", "area_udder",  "peri_udder",
          "exc_udder", "circ_udder","interval_sec"]]

X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.3,
random_state=42)
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [26]:
linearreg = LinearRegression()
forwad = SequentialFeatureSelector(
linearreg,
k_features=4,
forward=True,
verbose=1,
scoring="neg_mean_squared_error"
)
sf = forwad.fit(X_scaled,y)

Features: 4/4

In [24]:
feat_names = list(sf.k_feature_names_)
print(feat_names)

print(sf.k_feature_idx_)

['2', '4', '6', '7']
(2, 4, 6, 7)


In [35]:
import statsmodels.api as sm
X_opt = np.append(arr=np.ones((len(df),1)).astype(int), values = X_scaled[:,[2, 4, 6, 7]], axis=1)
regressor_OLS = sm.OLS(endog=y,exog=X_opt).fit()
regressor_OLS.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  yield   R-squared:                       0.630
Model:                            OLS   Adj. R-squared:                  0.591
Method:                 Least Squares   F-statistic:                     16.19
Date:                Sun, 19 Jan 2025   Prob (F-statistic):           8.03e-08
Time:                        10:50:59   Log-Likelihood:                -99.102
No. Observations:                  43   AIC:                             208.2
Df Residuals:                      38   BIC:                             217.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         18.6977      0.393     47.533      0.000      17.901      19.494
x1            -1.0484      0.426     -2.459      0.019      -1.912      -0.185
x2             0.4338      0.416      1.043      0.304      -0.409       1.276
x3            -0.3524      0.420     -0.839      0.407      -1.203       0.498
x4             3.1607      0.429      7.365      0.000       2.292       4.029
==============================================================================
Omnibus:                        2.268   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.322   Jarque-Bera (JB):                1.790
Skew:                          -0.499   Prob(JB):                        0.409
Kurtosis:                       2.961   Cond. No.                         1.59
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [34]:
var = np.array(["sarea_udder", "vol_udder", 
          "days_in_milk", "area_udder",  "peri_udder",
          "exc_udder", "circ_udder","interval_sec"])
var[[2,4,6,7]]

array(['days_in_milk', 'peri_udder', 'circ_udder', 'interval_sec'],
      dtype='<U12')